# QCoDeS example with Keithley 3706A System Switch

## Basic Operation

Let us, first, import QCoDeS driver for the Keithley System Switch:

In [1]:
import qcodes as qc
from qcodes.instrument_drivers.tektronix.Keithley_3706A_Matrix_channels import Keithley_3706A

Now, we create a station to hold our instrument:

In [2]:
station = qc.Station()

We finalize the initialization of the instrument by instantiation and then adding it to the station:

In [3]:
smatrix = Keithley_3706A('smatrix', address="USB0::0x05E6::0x3706::04447336::INSTR")
station.add_component(smatrix)

Connected to: KEITHLEY INSTRUMENTS INC. 3706A-SNFP SYSTEM SWITCH (serial:04447336, firmware:01.56a)
Slot 1- Model:3730, Matrix Type:6x16 High Density Matrix, Firmware:01.40h, Serial:4447332
Slot 2- Model:3730, Matrix Type:6x16 High Density Matrix, Firmware:01.40h, Serial:4398982
Slot 3- Model:3730, Matrix Type:6x16 High Density Matrix, Firmware:01.40h, Serial:4447333


'smatrix'

Let us, first, examine the properties of our instrument via its snapshot:

In [4]:
smatrix.print_readable_snapshot()

smatrix:
	parameter           value
--------------------------------------------------------------------------------
IDN                  :	{'vendor': 'KEITHLEY INSTRUMENTS INC.', 'model': '3706...
channel_connect_rule :	BREAK_BEFORE_MAKE 
close_channel        :	None 
exclusive_close      :	None 
exclusive_slot_close :	None 
gpib_address         :	16 
gpib_enable          :	True 
lan_enable           :	True 
open_channel         :	None 
reset_channel        :	None 
timeout              :	5 (s)


### Connection types

System switch can be connected to a setup through USB, LAN as well as GPIB. Depending on the needs, one can enable (disable) LAN and/or GPIB connections.

In [5]:
smatrix.gpib_enable(False)

In [6]:
smatrix.lan_enable(False)

In [7]:
smatrix.print_readable_snapshot()

smatrix:
	parameter           value
--------------------------------------------------------------------------------
IDN                  :	{'vendor': 'KEITHLEY INSTRUMENTS INC.', 'model': '3706...
channel_connect_rule :	BREAK_BEFORE_MAKE 
close_channel        :	None 
exclusive_close      :	None 
exclusive_slot_close :	None 
gpib_address         :	16 
gpib_enable          :	False 
lan_enable           :	False 
open_channel         :	None 
reset_channel        :	None 
timeout              :	5 (s)


In [8]:
smatrix.gpib_enable(True)

In [9]:
smatrix.lan_enable(True)

GPIB connection address can be changed at will. An integer value in between `1` and `30` can be assigned via:

In [10]:
smatrix.gpib_address(1)

In [11]:
smatrix.print_readable_snapshot()

smatrix:
	parameter           value
--------------------------------------------------------------------------------
IDN                  :	{'vendor': 'KEITHLEY INSTRUMENTS INC.', 'model': '3706...
channel_connect_rule :	BREAK_BEFORE_MAKE 
close_channel        :	None 
exclusive_close      :	None 
exclusive_slot_close :	None 
gpib_address         :	1 
gpib_enable          :	True 
lan_enable           :	True 
open_channel         :	None 
reset_channel        :	None 
timeout              :	5 (s)


If an invalid address is passed as an argument, QCoDeS shall rise an error and the present address shall be preserved.

In [12]:
smatrix.gpib_address(31)

ValueError: ('31 is invalid: must be between 1 and 30 inclusive; Parameter: smatrix.gpib_address', 'setting smatrix_gpib_address to 31')

If the preferred connection method is LAN, one can querry the IP address of the instrument through the driver.

In [13]:
smatrix.get_ip_address()

'0.0.0.0'

Here, the returned address is the default value if there is no LAN connection; indeed in this example we are connected to the instrument via USB. In cases where a reset to the LAN is required, a simple function call is sufficient:

In [14]:
smatrix.reset_local_network()

### Identifications

While QCoDeS prints out the instrument identifications including the installed matrix types upon a successfull connection, these metadata can be queried later on, as well.

In [15]:
smatrix.get_idn()

{'vendor': 'KEITHLEY INSTRUMENTS INC.',
 'model': '3706A-SNFP',
 'serial': '04447336',
 'firmware': '01.56a'}

In [16]:
smatrix.get_switch_cards()

({'slot_no': '1',
  'model': '3730',
  'mtype': '6x16 High Density Matrix',
  'firmware': '01.40h',
  'serial': '4447332'},
 {'slot_no': '2',
  'model': '3730',
  'mtype': '6x16 High Density Matrix',
  'firmware': '01.40h',
  'serial': '4398982'},
 {'slot_no': '3',
  'model': '3730',
  'mtype': '6x16 High Density Matrix',
  'firmware': '01.40h',
  'serial': '4447333'})

### Save and load setups, memory management

The system switch provides an option for the users to save their current setups to be used later. One can save a configuration either to the non-volatile memory of the instrument or to a USB drive. To this end, we use `save_setup()` function which either takes no arguments or a string which represents the path and the file name to which the setup shall be saved on an external drive. If no arguments provided, the setup shall be saved to the intstument. Note that, in the latter case, any existing saved configurations shall be overwritten. 

In [21]:
smatrix.save_setup()

We can recall saved setups via `load_setup()` function. It takes a single argument: If it is `0`, factory defaults load, if it is `1`, the saved setup from the nonvolatile memory is recalled. Otherwise, a string specifying the relative path to the saved setup on a USB drive should be passed in.

In [22]:
smatrix.load_setup(0)

In [23]:
smatrix.load_setup(1)

The system swicth has limited memory which can be queried via:

In [24]:
smatrix.get_available_memory()

{'System Memory  (%)': '89.72',
 'Script Memory  (%)': '100.00',
 'Pattern Memory (%)': '100.00',
 'Config Memory  (%)': '100.00'}

The memory usage of a saved setup is depicted by the `Config Memory`. User given channel patterns occupy the `Pattern Memory`. The system swicth is capable of storing and running Lua scripts. This functionality is not explicitly implemented by the QCoDeS driver. However, if desired, one can still send, save and execute Lua scripts through `write()` and `ask()` methods of the `InstrumentBase` class of QCoDeS which is inherited by every instrument driver. If this is the case, the saved Lua scripts occupy the `Script Memory`.    

## Channel Control and Manipulations

### Channel Specifiers

First, let us call the installed switch cards again:

In [25]:
smatrix.get_switch_cards()

({'slot_no': '1',
  'model': '3730',
  'mtype': '6x16 High Density Matrix',
  'firmware': '01.40h',
  'serial': '4447332'},
 {'slot_no': '2',
  'model': '3730',
  'mtype': '6x16 High Density Matrix',
  'firmware': '01.40h',
  'serial': '4398982'},
 {'slot_no': '3',
  'model': '3730',
  'mtype': '6x16 High Density Matrix',
  'firmware': '01.40h',
  'serial': '4447333'})

The system switch has six available slots from which three are currently occupied by `6x16` high density matrices. As implied, these matrices have `6` rows and `16` columns, hence, in total, `96` available channels, each. 

Each channel has identical properties but unique names. The properties of the channels can be manipulated via calling them individually, as a group or as a whole slot. The naming of the channels follows a general pattern. In particular, here for the model `3730` which has no `bank`, the names starts with the slot number, then the row number and finally follows the cloumn number which should always be two characters. For example, `1101` represents the channel at `Slot 1`, `Row 1` and `Column 1`. For other models, users should refer to the corresponding instrument manual.

In certain cases manual deduction of the name of the channel could be more efficient. On the other hand, user can easily query the available channels as follows:

In [26]:
smatrix.get_channels()

['1101',
 '1102',
 '1103',
 '1104',
 '1105',
 '1106',
 '1107',
 '1108',
 '1109',
 '1110',
 '1111',
 '1112',
 '1113',
 '1114',
 '1115',
 '1116',
 '1201',
 '1202',
 '1203',
 '1204',
 '1205',
 '1206',
 '1207',
 '1208',
 '1209',
 '1210',
 '1211',
 '1212',
 '1213',
 '1214',
 '1215',
 '1216',
 '1301',
 '1302',
 '1303',
 '1304',
 '1305',
 '1306',
 '1307',
 '1308',
 '1309',
 '1310',
 '1311',
 '1312',
 '1313',
 '1314',
 '1315',
 '1316',
 '1401',
 '1402',
 '1403',
 '1404',
 '1405',
 '1406',
 '1407',
 '1408',
 '1409',
 '1410',
 '1411',
 '1412',
 '1413',
 '1414',
 '1415',
 '1416',
 '1501',
 '1502',
 '1503',
 '1504',
 '1505',
 '1506',
 '1507',
 '1508',
 '1509',
 '1510',
 '1511',
 '1512',
 '1513',
 '1514',
 '1515',
 '1516',
 '1601',
 '1602',
 '1603',
 '1604',
 '1605',
 '1606',
 '1607',
 '1608',
 '1609',
 '1610',
 '1611',
 '1612',
 '1613',
 '1614',
 '1615',
 '1616',
 '2101',
 '2102',
 '2103',
 '2104',
 '2105',
 '2106',
 '2107',
 '2108',
 '2109',
 '2110',
 '2111',
 '2112',
 '2113',
 '2114',
 '2115',
 

The `get_channels()` function returns an array of strings each being the name of an available channel in all occupied slots. The return type being `string` is intentional as the instrument accepts the channel names as strings and not integers during any manipulation. Thus, any element(s) of this list can be safely passed as an argument to another function which specifies a channel property. 

If desired, the available channels in a given slot can be queried, as well.

In [27]:
smatrix.get_channels_by_slot(1)

['1101',
 '1102',
 '1103',
 '1104',
 '1105',
 '1106',
 '1107',
 '1108',
 '1109',
 '1110',
 '1111',
 '1112',
 '1113',
 '1114',
 '1115',
 '1116',
 '1201',
 '1202',
 '1203',
 '1204',
 '1205',
 '1206',
 '1207',
 '1208',
 '1209',
 '1210',
 '1211',
 '1212',
 '1213',
 '1214',
 '1215',
 '1216',
 '1301',
 '1302',
 '1303',
 '1304',
 '1305',
 '1306',
 '1307',
 '1308',
 '1309',
 '1310',
 '1311',
 '1312',
 '1313',
 '1314',
 '1315',
 '1316',
 '1401',
 '1402',
 '1403',
 '1404',
 '1405',
 '1406',
 '1407',
 '1408',
 '1409',
 '1410',
 '1411',
 '1412',
 '1413',
 '1414',
 '1415',
 '1416',
 '1501',
 '1502',
 '1503',
 '1504',
 '1505',
 '1506',
 '1507',
 '1508',
 '1509',
 '1510',
 '1511',
 '1512',
 '1513',
 '1514',
 '1515',
 '1516',
 '1601',
 '1602',
 '1603',
 '1604',
 '1605',
 '1606',
 '1607',
 '1608',
 '1609',
 '1610',
 '1611',
 '1612',
 '1613',
 '1614',
 '1615',
 '1616']

### Backplanes Specifiers

Each matrix has typically six analog backplane relays which can be associated which a channel(s). The naming scheme of the relays are similar to that of the channels. Each name start with the slot number, continues with `9`, which is the unique backplane number, and then finishes with backplane relay identifier which can take the values `11`, `12`, ..., `16`. That is, we can select the backplane relay `12` on `Slot 3` via `3912`. 

Unless a backplane is associated with a channel, the instrument does not return the names of the relays. This is overcomed within the QCoDeS driver, so that user can list the available analog backplane relays.

In [28]:
smatrix.get_analog_backplane_specifiers()

['1911',
 '1912',
 '1913',
 '1914',
 '1915',
 '1916',
 '2911',
 '2912',
 '2913',
 '2914',
 '2915',
 '2916',
 '3911',
 '3912',
 '3913',
 '3914',
 '3915',
 '3916']

### Control and manipulations